#### Initial setup

In [ ]:
# Project info
PROJECT_ID = 'marvin-voice'
GIT_OWNER, GIT_REPO = 'patrikkj', 'marvin-models'
DATASET = 'dataset'

# Set current working directory to project root
import os
if os.getcwd().endswith("notebooks"):
    %cd -q ..
    
# Identify environment
IN_COLAB = 'google.colab' in str(get_ipython())

In [ ]:
# Run this cell only if we're running from Google Colab
if IN_COLAB:
    !git clone -q https://github.com/{GIT_OWNER}/{GIT_REPO}.git
    !gsutil -q cp gs://{PROJECT_ID}/data/raw/{DATASET}.tar.gz /tmp
    !tar -xf /tmp/{DATASET}.tar.gz -C /content/{GIT_REPO}/data
    import sys
    sys.path.append(f"/content/{GIT_REPO}")
else:
    !gsutil -q cp gs://{PROJECT_ID}/data/raw/{DATASET}.tar.gz data
    !tar -xf data/{DATASET}.tar.gz -C data

In [ ]:
# Internal modules
import os, sys, glob
from importlib import reload

# External modules
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_io as tfio
from tensorflow_io import experimental as tfex

# Colab modules
if IN_COLAB:
    from google.colab import auth
    from IPython import display
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    
# Scripts
from scripts import preprocessing

In [ ]:
preprocessing = reload(preprocessing)

paths = glob.glob(f"data/dataset/*/*.wav")
paths = paths[:500] + paths[-500:]

tensors = [preprocessing.to_tensor(path) for path in paths]
specs = [preprocessing.tensor_to_log_mel_spec(tensor) for tensor in tensors]
mfccs = [preprocessing.batch_log_mel_spec_to_mfccs(spec) for spec in specs]

labels = [preprocessing.to_binary_label(path) for path in paths]
#dataset = tf.data.Dataset.from_tensor_slices((specs, labels))


In [ ]:
preprocessing = reload(preprocessing)

paths = glob.glob(f"data/dataset/*/*.wav")

tensors_arr = np.array([preprocessing.to_tensor(path) for path in paths])
labels_arr = np.array([preprocessing.to_binary_label(path) for path in paths])

print(tensors_arr.shape)
print(labels_arr.shape)

In [ ]:
import numpy as np
print(np.array(tensors))

In [ ]:
# Defines ratios, w.r.t. whole dataset.
ratio_train = 0.8
ratio_val = 0.1
ratio_test = 0.1

# Produces test split.
x_remaining, x_test, y_remaining, y_test = train_test_split(
    x, y, test_size=test_ratio)

# Adjusts val ratio, w.r.t. remaining dataset.
ratio_remaining = 1 - ratio_test
ratio_val_adjusted = ratio_val / ratio_remaining

# Produces train and val splits.
x_train, x_val, y_train, y_val = train_test_split(
    x_remaining, y_remaining, test_size=ratio_val_adjusted)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((mfccs, labels))
"""
for tensor, label in dataset.take(5):
    print(tensor.shape)
    print(label.shape)
    print()
"""
dataset = dataset.shuffle(10000).batch(512)

model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(dataset, epochs=50)

In [ ]:
from scripts import record_audio
audio, sr = record_audio.get_audio()